In [ ]:
from aux import *

In [ ]:
# The sequence of protein to fold
# prot_seq = "YYDPETGTWY"
prot_seq = "YYDPET"

In [ ]:
prot_seq, q, energy_expr = prepare_quantum(prot_seq)

In [ ]:
energy_func = lambdify(q, energy_expr)

In [ ]:
sched = np.power(2, np.linspace(4, -1, 300))
max(sched), min(sched), len(sched)

In [ ]:
plt.plot(sched)
plt.show()

In [ ]:
plt.plot(np.exp(-10/sched))
plt.show()

In [ ]:
num_cores = 12

res = Parallel(n_jobs=num_cores)(delayed(simulate_quantum)(prot_seq, q, energy_expr, sched, jobid) for jobid in range(720))

In [ ]:
tra, H, E, seeds, results = zip(*res)

In [ ]:
tra = np.array(tra)
history = np.array(H)
energy = np.array(E)

In [ ]:
acp = history.cumsum(axis=1)

In [ ]:
plt.plot(acp.mean(axis=0))
plt.fill_between(
    range(acp.shape[1]),
    acp.mean(axis=0) - acp.std(axis=0),
    acp.mean(axis=0) + acp.std(axis=0), alpha=0.2)
plt.show()

In [ ]:
enes = [energy_func(*qs) for qs in results]
best = np.argmin(enes)

In [ ]:
make_viz(tra[best], prot_seq, schedule=sched, movie=True, energy_func=energy_func)

In [ ]:
all_best = np.array(enes) == min(enes)
(all_best).sum()

In [ ]:
plt.plot(np.sort(enes))
plt.show()

In [ ]:
rnd_best = np.random.choice(np.argwhere(all_best).reshape((-1,)))
print(rnd_best)
plt.plot(energy[rnd_best])
plt.ylim(min(enes)-5, 200)
plt.show()

In [ ]:
show_video()

In [ ]:
def preprocess_expr(H):
    # Helper function to clean resulting expressions
    # some zero coefficients fail to evaluate to 0, rounding is necessary
    # for all qbits, qi**n = qi. bc qi in [0,1]
    H = expand(H).evalf()
    tmp = H
    for a in preorder_traversal(tmp):
        if isinstance(a, Float):
            H = H.subs(a, round(a, 8))
    degr = Poly(H).degree()
    for qi in q:
        for n in reversed(range(2, degr + 1)):
            H = H.subs(qi**n, qi)
    return H


In [ ]:
energy_expr = preprocess_expr(energy_expr)

In [ ]:
energy_expr